# **Sentiment Analysis(YouTube)**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
# fasttext.util.download_model('ko', if_exists='ignore')   # FastText 모델 사용 시에만 필요
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

## **Read Data**

### **① KOSELF + Blog 감성사전**

In [2]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

In [3]:
len(positive), len(positive_blog), len(negative), len(negative_blog)

(48, 46, 47, 74)

In [4]:
# 긍정사전 통합
for i in range(len(positive_blog)):
    if positive_blog[i] not in positive:
        positive.append(positive_blog[i])

# 부정사전 통합
for i in range(len(negative_blog)):
    if negative_blog[i] not in negative:
        negative.append(negative_blog[i])

In [5]:
len(positive), len(negative)

(92, 111)

In [6]:
# 중복 확인
cnt = 0
for i in range(len(positive)):
    if positive[i] in negative:
        cnt += 1

cnt

0

### **② Text Data**

In [7]:
# 종목별, 연도별 데이터 통합
youtube = pd.read_csv('../../../../Code/result_youtube_token.csv')

youtube

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,token
0,하이닉스,660,한국경제TV,2021091600,삼성전자·SK하이닉스 비중확대 해야 할 이유(노근창)/ 주식경제 이슈분석 / 한국경제TV,4오랄3십일조를판외국인의지금반도체를향해서는이달들어서조금씩매수를해주고있습니다어떻게해석...,472,941,○ 노근창 (현대차증권 리서치센터장)\n○ 곽수종 앵커\n○ 손현정 앵커\n\n\n...,https://www.youtube.com/watch?v=WRdlEVeBsCs,"['파마', '게임', '겨울', '적로', '현재', '트레인', '걱정', '살..."
1,하이닉스,660,한국경제TV,2021081300,"삼성전자·SK하이닉스 급락 떨어지는 칼날 잡아라? / 삼성전자, SK하이닉스, 원달...",출발중시오늘도진짜전략로시작하게있는데오늘은타임경제이순조대표모셨습니다어서오세약간64일단...,339119,1268,앵커 : 하경민\n \n출연자 : 이승조 다인경제 대표\n\n시장이 중요힌 변곡점에...,https://www.youtube.com/watch?v=x7to5CTAU4I,"['경제', '라면', '그룹', '바이오', '심절', '인벌스', '배하', '..."
2,하이닉스,660,한국경제TV,2021081700,삼성전자·SK하이닉스 외국인 매도 이유 분석 / 김선형 L&S 홀딩스 대표 / 증시...,46시장의이슈속에서중장기적로가져가며4종목때그리고포트폴리오전략을함께세워보도록하겠습니다...,22264,569,진행 : 이경은 앵커\n출연 : 김선형 L&S 홀딩스 대표\n방송일 : 2021년 ...,https://www.youtube.com/watch?v=Rmqk3OXijGw,"['수술', '즉석', '고서', '지속', '적로', '내용', '갈등', '반발..."
3,하이닉스,660,한국경제TV,2021061400,"주가는 횡보지만, SK하이닉스 임원들은 매수 행렬 / 공진단 / 한국경제TV",톡톡놈놈놈놈공시의진단을해보는시간입니다타키온뉴스에조호진대표입니다안녕하세그러세4오늘은하...,6881,177,앵커 : 하경민\n\n출연자 : 조호진 타키온뉴스 대표 \n\n△타키온 뉴스 구독\...,https://www.youtube.com/watch?v=vJkyrt2-avs,"['동공', '스포츠', '임계', '오늘', '보유', '공신', '뭔가', '하..."
4,하이닉스,660,한국경제TV,2021081200,"모건스탠리 ""삼성전자, SK하이닉스 팔아라"" 수급전쟁 속 실적 좋은 이 기업 / 삼...",이번프로마케터시간인데시장본을시장을보는안목을키우는그런시간입니다오늘은증권중소성장기업분석...,13570,619,▶ 출연자\n최성민 앵커 / 글로벌 투자 전략가\n윤혁진 / SK증권 연구위원\n\...,https://www.youtube.com/watch?v=MXO7e-54XgA,"['정도', '오늘', '스토리', '체적', '추정', '신드', '과정', '제..."
...,...,...,...,...,...,...,...,...,...,...,...
6380,셀트리온,68270,슈카월드,2019061900,'양적 완화'란 무엇인가? 우리가 해야할 일은?,르3메인얘기는양적만합니다양쪽은얘기를왜하느냐끝날때쯤에들어오시면알게될거에장점와나를알기전...,747665,1681,요즘 핫한 키워드 '양적 완화'를 알아봅시다.\n쉽고 재미있는 슈카의 경제상식 썰!...,https://www.youtube.com/watch?v=oelFn7E0fqY,NaN
6381,셀트리온,68270,슈카월드,2019051200,"[레전썰] 사상 최악의 주가 조작 사건, ""루보"" 사태",니오기까지는이해를할수있잖여기서격이도그래도좀인간적인집에서그래도나여기서부터약을먹지않고서...,804287,1037,다시 태어난 레전(前)썰\n못보신 분들을 위해 깔끔하게 재편집했습니다.\n4편 - ...,https://www.youtube.com/watch?v=nwqnvvUU-MA,NaN
6382,셀트리온,68270,슈카월드,2019040300,현대차에 1조원 물리고 존버중인 엘리엇 근황,가장유명한얘기부터드릴께이해를좀쉽게해야되니까2003년에있었던그룹의소버린공격사태입니다이...,663480,1624,엘리엇썰 1부 풀영상 바로가기↓\nhttps://www.youtube.com/wat...,https://www.youtube.com/watch?v=g8MJDRovq4g,NaN
6383,셀트리온,68270,슈카월드,2019072600,액티브 투자 vs 패시브 투자 당신의 선택은?,르3방송을하다보면은듣는이렇게듣는말니면많이뜨는채팀니면많이뜬곱뭐라그러냐댓글같은겠어그1번...,290486,1888,액티브 vs 패시브 당신의 성향은 무엇입니까?\n투자의 기초(?)에 대해 Arabo...,https://www.youtube.com/watch?v=Nzjht2z3iqE,NaN


## **Sentiment Score**

In [8]:
youtube[youtube['token'].isna()]

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,token
3123,현대차,5380,삼프로TV_경제의신과함께,2021030200,현대차그룹의 야심작 '아이오닉5'.. 완판에 이어 증산까지?_오늘아침 page2_권...,NaN,356237,7095,삼프로TV주식대학봄학기염블리의친절한주식기초교육미주미장우석본부장이항영대표의미국주식완전...,https://www.youtube.com/watch?v=rrt8SrE1AS4,NaN
3124,현대차,5380,삼프로TV_경제의신과함께,2021010800,애플과 현대차의 애플카 협업 이슈.. 오늘은 '자동차의 날'일까? f.이베스트투자증...,산업을이해하고기업을이해해야돈의흐름이보입니다리서치명과메리츠증권과3%과함께야심차게준비한...,290205,2044,삼프로TV가준비한다양한프리미엄콘텐츠삼프로TV주식대학업종기초교육과실전투자전략https...,https://www.youtube.com/watch?v=_wcOsuS6Auk,NaN
3125,현대차,5380,삼프로TV_경제의신과함께,2021072100,[퇴근길 라이브&백브리핑 라이브] 전기·수소 급변하는 시기 현대차의 위치는? / 카...,투를사람들의진짜대화32퇴근깰라이브반갑습니다저는점프로안녕하세저는염성하니삽니다네맞습니다...,331599,10055,삼프로TV주식대학여름학기삼프로TV주식대학박병창의매매의기술염승환의친절한주식기초교육강의...,https://www.youtube.com/watch?v=GT7N-Go4Li8,NaN
3126,현대차,5380,삼프로TV_경제의신과함께,2021012600,현대차그룹.. 테슬라와 차별화된 미래 가치를 말한다 f.현대차증권 장문수 연구위원,산업을이해하고기업을이해해야돈의흐름이보입니다리서치명과메리츠증권과3%과함께야심차게준비한...,135506,2426,삼프로TV주식대학업종기초교육과실전투자전략httpswwwpage2krconcert20...,https://www.youtube.com/watch?v=bs0YTCbsLwU,NaN
3127,현대차,5380,삼프로TV_경제의신과함께,2021090700,[뉴스3] 현대차 2045년 탄소배출 '제로' 첫 선언..전동화·그린수소 핵심축,그러면그뉴스3초지뉴스부터전해드리도록하겠습니다한번적들이미루는많이해주시길한다그래모시단어...,14564,1286,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=FvPX_Wqff1k,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6380,셀트리온,68270,슈카월드,2019061900,'양적 완화'란 무엇인가? 우리가 해야할 일은?,르3메인얘기는양적만합니다양쪽은얘기를왜하느냐끝날때쯤에들어오시면알게될거에장점와나를알기전...,747665,1681,요즘 핫한 키워드 '양적 완화'를 알아봅시다.\n쉽고 재미있는 슈카의 경제상식 썰!...,https://www.youtube.com/watch?v=oelFn7E0fqY,NaN
6381,셀트리온,68270,슈카월드,2019051200,"[레전썰] 사상 최악의 주가 조작 사건, ""루보"" 사태",니오기까지는이해를할수있잖여기서격이도그래도좀인간적인집에서그래도나여기서부터약을먹지않고서...,804287,1037,다시 태어난 레전(前)썰\n못보신 분들을 위해 깔끔하게 재편집했습니다.\n4편 - ...,https://www.youtube.com/watch?v=nwqnvvUU-MA,NaN
6382,셀트리온,68270,슈카월드,2019040300,현대차에 1조원 물리고 존버중인 엘리엇 근황,가장유명한얘기부터드릴께이해를좀쉽게해야되니까2003년에있었던그룹의소버린공격사태입니다이...,663480,1624,엘리엇썰 1부 풀영상 바로가기↓\nhttps://www.youtube.com/wat...,https://www.youtube.com/watch?v=g8MJDRovq4g,NaN
6383,셀트리온,68270,슈카월드,2019072600,액티브 투자 vs 패시브 투자 당신의 선택은?,르3방송을하다보면은듣는이렇게듣는말니면많이뜨는채팀니면많이뜬곱뭐라그러냐댓글같은겠어그1번...,290486,1888,액티브 vs 패시브 당신의 성향은 무엇입니까?\n투자의 기초(?)에 대해 Arabo...,https://www.youtube.com/watch?v=Nzjht2z3iqE,NaN


In [9]:
# 결측치 제거
youtube.dropna(inplace=True)
youtube = youtube.reset_index(drop=True)
youtube.shape

(3076, 11)

In [10]:
# iteration limit 조정
limit_number = 15000
sys.setrecursionlimit(limit_number)

youtube['Positive_Score'] = 0
youtube['Negative_Score'] = 0
youtube['Ratio'] = 0.1
youtube['Pred'] = 0
youtube['NSI'] = 0.1

for i in range(len(youtube)):
    pos_score = 0 ; neg_score = 0
    
    for j in range(len(youtube['token'][i])):
        if youtube['token'][i][j] in positive:
            pos_score += 1
        elif youtube['token'][i][j] in negative:
            neg_score += 1
        else:
            pass
    
    youtube['Positive_Score'][i] = pos_score
    youtube['Negative_Score'][i] = neg_score
    
    # 긍정과 부정의 비율
    if (pos_score==0) and (neg_score==0):
        youtube['Ratio'][i] = 0.5
    else:
        youtube['Ratio'][i] = pos_score / (pos_score + neg_score)
    
    # 예측 결과
    if youtube['Ratio'][i]>=0.5:
        youtube['Pred'][i] = 1
    else:
        youtube['Pred'] = -1
    
    # 뉴스심리지수(NSI) 계산
    if (pos_score==0) and (neg_score==0):
        youtube['NSI'][i] = 101
    else:
        youtube['NSI'][i] = (pos_score - neg_score) / (pos_score + neg_score) * 100 + 100

In [11]:
youtube

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,token,Positive_Score,Negative_Score,Ratio,Pred,NSI
0,하이닉스,660,한국경제TV,2021091600,삼성전자·SK하이닉스 비중확대 해야 할 이유(노근창)/ 주식경제 이슈분석 / 한국경제TV,4오랄3십일조를판외국인의지금반도체를향해서는이달들어서조금씩매수를해주고있습니다어떻게해석...,472,941,○ 노근창 (현대차증권 리서치센터장)\n○ 곽수종 앵커\n○ 손현정 앵커\n\n\n...,https://www.youtube.com/watch?v=WRdlEVeBsCs,"['파마', '게임', '겨울', '적로', '현재', '트레인', '걱정', '살...",0,0,0.5,1,101.0
1,하이닉스,660,한국경제TV,2021081300,"삼성전자·SK하이닉스 급락 떨어지는 칼날 잡아라? / 삼성전자, SK하이닉스, 원달...",출발중시오늘도진짜전략로시작하게있는데오늘은타임경제이순조대표모셨습니다어서오세약간64일단...,339119,1268,앵커 : 하경민\n \n출연자 : 이승조 다인경제 대표\n\n시장이 중요힌 변곡점에...,https://www.youtube.com/watch?v=x7to5CTAU4I,"['경제', '라면', '그룹', '바이오', '심절', '인벌스', '배하', '...",0,0,0.5,1,101.0
2,하이닉스,660,한국경제TV,2021081700,삼성전자·SK하이닉스 외국인 매도 이유 분석 / 김선형 L&S 홀딩스 대표 / 증시...,46시장의이슈속에서중장기적로가져가며4종목때그리고포트폴리오전략을함께세워보도록하겠습니다...,22264,569,진행 : 이경은 앵커\n출연 : 김선형 L&S 홀딩스 대표\n방송일 : 2021년 ...,https://www.youtube.com/watch?v=Rmqk3OXijGw,"['수술', '즉석', '고서', '지속', '적로', '내용', '갈등', '반발...",0,0,0.5,1,101.0
3,하이닉스,660,한국경제TV,2021061400,"주가는 횡보지만, SK하이닉스 임원들은 매수 행렬 / 공진단 / 한국경제TV",톡톡놈놈놈놈공시의진단을해보는시간입니다타키온뉴스에조호진대표입니다안녕하세그러세4오늘은하...,6881,177,앵커 : 하경민\n\n출연자 : 조호진 타키온뉴스 대표 \n\n△타키온 뉴스 구독\...,https://www.youtube.com/watch?v=vJkyrt2-avs,"['동공', '스포츠', '임계', '오늘', '보유', '공신', '뭔가', '하...",0,0,0.5,1,101.0
4,하이닉스,660,한국경제TV,2021081200,"모건스탠리 ""삼성전자, SK하이닉스 팔아라"" 수급전쟁 속 실적 좋은 이 기업 / 삼...",이번프로마케터시간인데시장본을시장을보는안목을키우는그런시간입니다오늘은증권중소성장기업분석...,13570,619,▶ 출연자\n최성민 앵커 / 글로벌 투자 전략가\n윤혁진 / SK증권 연구위원\n\...,https://www.youtube.com/watch?v=MXO7e-54XgA,"['정도', '오늘', '스토리', '체적', '추정', '신드', '과정', '제...",0,0,0.5,1,101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071,현대차,5380,삼프로TV_경제의신과함께,2021060700,"[심층 인터뷰] 현대차그룹, 긍정적 지표로 기업 가치 올린다? f.머니투데이방송 권...",작문을올해일에남머니투데이방송에건수를기님모시도록하겠습니다어서오십시오안녕하세네이트때방송...,24941,1657,삼프로TV주식대학봄학기염블리의친절한주식기초교육미주미장우석본부장이항영대표의미국주식완전...,https://www.youtube.com/watch?v=StYCnsy1WG0,"['스토', '먼저', '엑셀', '작문', '클릭', '테슬라', '바이오', '...",0,0,0.5,1,101.0
3072,현대차,5380,삼프로TV_경제의신과함께,2020111300,최대 주주의 시각에서 바라보는 현대차 그룹의 '지배구조 개편'.. 주가의 흐름은?_...,알이용정정보도가충북농협이관리하는충청부터농산물공동브랜드알이샴농식품부과수분야연차평가에서...,191334,7312,0000오프닝0533뉴스3아시아나대한항공품으로날아드나초대형항공사탄생예고2430US각...,https://www.youtube.com/watch?v=bhLks1Wu1Jg,"['이엄', '고삼', '웃기', '제초', '테슬라', '농산물', '에너지', ...",0,0,0.5,1,101.0
3073,현대차,5380,삼프로TV_경제의신과함께,2019112800,현대차 동남아 진출! 성공할 수 있을까?,하토주권에김지루의애널리스트와동차얘기좀하고싶어서오십시오네안녕하세오랜만에뵙겠습니다얼마안...,3330,2818,프리미엄교육자세히알아보기클릭httpswwwpage2krnotice77신과함께가파헤친...,https://www.youtube.com/watch?v=BxpTCNfwGlA,"['제휴', '스터디', '드하', '안주', '고어', '먼저', '미쓰비시', ...",0,0,0.5,1,101.0
3074,현대차,5380,삼프로TV_경제의신과함께,2021062900,"현대차그룹의 주가가 상승하기 위한 조건은?_오늘아침 page2_유지웅, 박병창, 염승환",55222잘나가는그해서는뭔가바루더모바일쿠폰할인1최대12%없길6대상품권보유에견적서거래...,312035,7355,삼프로TV주식대학여름학기삼프로TVx메리츠증권하반기투자전략과업종별시장전망6월구매한정5...,https://www.youtube.com/watch?v=y5xCPfT4gCE,"['보합', '당일', '테슬라', '에너지', '컨센서스', '다임러', '바이오...",0,0,0.5,1,101.0


In [12]:
youtube[youtube['Positive_Score']!=0]

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,token,Positive_Score,Negative_Score,Ratio,Pred,NSI


In [13]:
youtube[youtube['Negative_Score']!=0]

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,token,Positive_Score,Negative_Score,Ratio,Pred,NSI


In [14]:
ensemble = pd.DataFrame({
    'code': youtube['st_cd'],
    'date': youtube['date'],
    'lstm': 0,
    'arima': 0,
    'fbprophet': 0,
    'news_text': 0.1,
    'youtube_text': youtube['Ratio'],
    'nsi': youtube['NSI'],
    'close': 0.1
})

In [15]:
ensemble

,code,date,lstm,arima,fbprophet,news_text,youtube_text,nsi,close
0,660,2021091600,0,0,0,0.1,0.5,101.0,0.1
1,660,2021081300,0,0,0,0.1,0.5,101.0,0.1
2,660,2021081700,0,0,0,0.1,0.5,101.0,0.1
3,660,2021061400,0,0,0,0.1,0.5,101.0,0.1
4,660,2021081200,0,0,0,0.1,0.5,101.0,0.1
...,...,...,...,...,...,...,...,...,...
3071,5380,2021060700,0,0,0,0.1,0.5,101.0,0.1
3072,5380,2020111300,0,0,0,0.1,0.5,101.0,0.1
3073,5380,2019112800,0,0,0,0.1,0.5,101.0,0.1
3074,5380,2021062900,0,0,0,0.1,0.5,101.0,0.1


In [16]:
# CSV로 저장
ensemble.to_csv('../../../../Final Data/ensemble_test_youtube.csv')